#### Firstly we need to install necessary packeges using *pip*

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

## Loading Dataset

In [ ]:
dataset_path='risqaliyevds/uzbek_ner'
dataset=load_dataset(dataset_path)
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.05k [00:00<?, ?B/s]

uzbek_ner.json:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19609 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ner', 'text'],
        num_rows: 19609
    })
})

In [ ]:
dataset['train'][1]['ner']

{'CARDINAL': None,
 'DATE': [''],
 'EMAIL': None,
 'EVENT': None,
 'FAC': None,
 'FACILITY': None,
 'GPE': ['O‘zbekiston', 'Suriya', 'AQSh', 'Vashington'],
 'JCH-2022': None,
 'JOURNAL': None,
 'LANGUAGE': None,
 'LAW': None,
 'LOC': ['Suriya', 'Shayrat aerodromi'],
 'MISC': None,
 'MONEY': None,
 'NORP': None,
 'ORDINAL': None,
 'ORG': ['AQSh qurolli kuchlari'],
 'PER': None,
 'PERCENT': None,
 'PERIOD': None,
 'PERSON': ['Rajab Toyyib Erdo‘g‘an', 'Jan-Klod Yunker'],
 'PHONE': None,
 'PRODUCT': None,
 'QUANTITY': None,
 'RASUM': None,
 'SOCIAL_MEDIA': None,
 'TIME': None,
 'WEBSITE': None,
 'WORK_OF_ART': None}

In [ ]:
dataset['train'][0]['text']

"Shvetsiya hukumati Stokholmdagi asosiy piyodalar ko‘chasi Drottninggatanda odamlar ustiga yuk mashinasini haydab borgani gumon qilinayotgan shaxsni qo‘lga oldi. Bu haqda Expressen TV efirida Shvetsiya bosh vaziri Stefan Lyoven ma'lum qildi. Ushbu shaxs surati ijtimoiy tarmoqda tarqalmoqda. U Spendrups kompaniyasiga tegishli yuk mashinasini o‘g‘irlagani aytilmoqda.   Oxirgi ma'lumotlarga ko‘ra, ushbu hujum oqibatida halok bo‘lganlar soni 5 nafarga yetdi, jarohatlanganlar soni aniq emas."

Dataset includes many named entity categories, but we will use just 5 of them:
*   Person Names
*   Dates
*   Locations
*   Organizations
*   Event

In [ ]:
print(list(dataset['train'].features['ner'].keys()))

['CARDINAL', 'DATE', 'EMAIL', 'EVENT', 'FAC', 'FACILITY', 'GPE', 'JCH-2022', 'JOURNAL', 'LANGUAGE', 'LAW', 'LOC', 'MISC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PER', 'PERCENT', 'PERIOD', 'PERSON', 'PHONE', 'PRODUCT', 'QUANTITY', 'RASUM', 'SOCIAL_MEDIA', 'TIME', 'WEBSITE', 'WORK_OF_ART']


## Dataset Preparation

Dataset contains some symbols to change, so we will replace these. Also we expand dataset with spitted text as word

In [ ]:
# ‘ , « , » , \xa0,
def symbol_change(text):
  text=text.replace('‘',"'").replace('“','"').replace('”','"').replace('«','"').replace('»','"').replace('\xa0',' ')
  text=text.replace('"',' " ').replace('.',' . ').replace(','," , ")
  return text

def symbol_change_for_ner(text):
  text=text.replace('‘',"'").replace('“','"').replace('”','"').replace('«','"').replace('»','"').replace('\xa0',' ')
  return text

In [ ]:
def replace_symbols(samples):

  prepared_text=[symbol_change(text) for text in samples['text']]

  samples['words']=[text.split() for text in prepared_text]
  return samples

In [ ]:
prep_dataset=dataset.map(replace_symbols,batched=True)

Map:   0%|          | 0/19609 [00:00<?, ? examples/s]

In [ ]:
prep_dataset['train'][0]['text']

"Shvetsiya hukumati Stokholmdagi asosiy piyodalar ko‘chasi Drottninggatanda odamlar ustiga yuk mashinasini haydab borgani gumon qilinayotgan shaxsni qo‘lga oldi. Bu haqda Expressen TV efirida Shvetsiya bosh vaziri Stefan Lyoven ma'lum qildi. Ushbu shaxs surati ijtimoiy tarmoqda tarqalmoqda. U Spendrups kompaniyasiga tegishli yuk mashinasini o‘g‘irlagani aytilmoqda.   Oxirgi ma'lumotlarga ko‘ra, ushbu hujum oqibatida halok bo‘lganlar soni 5 nafarga yetdi, jarohatlanganlar soni aniq emas."

In [ ]:
prep_dataset['train'][0]['words'][:10]

['Shvetsiya',
 'hukumati',
 'Stokholmdagi',
 'asosiy',
 'piyodalar',
 "ko'chasi",
 'Drottninggatanda',
 'odamlar',
 'ustiga',
 'yuk']

We will divide each entity lable as beginning (B-) and inside (I-). For instance, **Shavkat Mirziyoyev -> [ B-PERSON , I-PERSON ]**

In [ ]:
actual_ner_tags=['PERSON','GPE','ORG','LOC','DATE','EVENT']
new_ner_tags=['O','B-PERSON','B-GPE','B-ORG','B-LOC','B-DATE','B-EVENT']

def words_to_ner_tags(sample_ner,sample_words,actual_ner_tags=actual_ner_tags,new_ner_tags=new_ner_tags):
  new_words_tag=['O']*len(sample_words)

  for tag in actual_ner_tags:
    if sample_ner[tag] is not None:
      for tag_words in sample_ner[tag]:

          prep_tag_words=symbol_change_for_ner(tag_words)
          splitted_tag_words=prep_tag_words.split()

          for tag_word in splitted_tag_words:
            for i,word in enumerate(sample_words):
              if tag_word.lower() in word.lower() and word.lower().startswith(tag_word.lower()[0])  :
                if new_words_tag[i]=='O':
                  new_words_tag[i]='B-'+tag

  new_words_tag_label=[new_ner_tags.index(label) for label in new_words_tag]
  return new_words_tag_label

def create_words_ner(samples):
  samples['new_word_ner_tags']=[words_to_ner_tags(sample_ner,sample_words)
          for sample_ner,sample_words in zip(samples['ner'],samples['words'])]
  return samples
new_dataset=prep_dataset.map(create_words_ner,batched=True)

Map:   0%|          | 0/19609 [00:00<?, ? examples/s]

In [ ]:
words = new_dataset["train"][7]["words"]
labels = new_dataset["train"][7]["new_word_ner_tags"]
line1 = ""
line2 = ""
label_names=new_ner_tags.copy()
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

NATO  bosh kotibi Yyens    Stoltenberg AQSh  tomonidan Suriyadagi aviabazaga raketa zarbalari yo'llangani uchun mas'uliyatni mamlakat rahbariyati zimmasiga yukladi . " O'nlab kishilar hamda bolalarning o'limiga sabab bo'lgan Xan-Sheyxundagi dahshatli kimyoviy hujum ortidan AQSh  Suriyadagi Shayrat aerodromiga hujum qildi . Suriyadagi rejim bu hodisa uchun to'la javobgar " , — deyilgan alyans saytida e'lon qilingan bayonot matnida . Qayd etilishicha , NATO  Suriyadagi siyosiy barqarorlik o'rnatilishiga qaratilgan barcha harakatlarni qo'llab-quvvatlaydi . Bundan oldinroq AQSh  Stoltenbergni bo'lajak hujum haqida oldindan xabardor qilgani ma'lum bo'lgandi . 
B-ORG O    O      B-PERSON B-PERSON    B-GPE O         B-GPE      O          O      O         O           O     O            O        O           O         O       O O O      O        O     O           O        O     O       B-LOC           O         O        O     O       B-GPE B-GPE      B-LOC   B-LOC       O     O     O B-GPE      

## Pushing to Huggingface

Finally, we push dataset contains only words and ner tags to huggingface

In [ ]:
new_dataset=new_dataset.remove_columns(['text','ner'])
new_dataset=new_dataset.rename_column('new_word_ner_tags','ner_tags')
new_dataset

DatasetDict({
    train: Dataset({
        features: ['words', 'ner_tags'],
        num_rows: 19609
    })
})

In [ ]:
new_dataset['train'].features

{'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
from datasets import Dataset, Features, Sequence, ClassLabel, Value

ner_feature = ClassLabel(names=new_ner_tags)

final_dataset = new_dataset.cast_column("ner_tags", Sequence(ner_feature))
print(final_dataset)
print(final_dataset['train'].features)

Casting the dataset:   0%|          | 0/19609 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['words', 'ner_tags'],
        num_rows: 19609
    })
})
{'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PERSON', 'B-GPE', 'B-ORG', 'B-LOC', 'B-DATE', 'B-EVENT'], id=None), length=-1, id=None)}


Log in to Huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_id = "ShakhzoDavronov/ner-prepared-uzbek"
final_dataset.push_to_hub(repo_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/632 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ShakhzoDavronov/ner-prepared-uzbek/commit/37d471eff3983a22fae9589982808e32624c7318', commit_message='Upload dataset', commit_description='', oid='37d471eff3983a22fae9589982808e32624c7318', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ShakhzoDavronov/ner-prepared-uzbek', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ShakhzoDavronov/ner-prepared-uzbek'), pr_revision=None, pr_num=None)